# In the Name of God
Name : Mohammad Mohajel Sadegi

SID  : 810199483


# Initial classes:

## DEFINES:

In [1]:
BFS_PRINT_PATH = 0

## Abstract:
Implementation of BFS, IDS and A* search 

## Importing libraries:

In [2]:
import copy
from collections import Counter
from queue import Queue
import time
import numpy as np

## Class state

In [3]:
class State:
    def __init__(self, rocky_nodes) -> None:
        self.position = 0
        self.recipes_seen = [] #be very carefull changed
        self.morids_seen = []
        self.prev_state = None
        self.rocky_nodes = dict.fromkeys(rocky_nodes, 0)
        self.rocky_remain = 0
        self.cost = 0

    def __members(self):
        return (self.position, self.recipes_seen, self.morids_seen, self.rocky_remain) #delete self.cost


    def __eq__(self, other):
        if type(other) is type(self):
            return self.__members() == other.__members()
        return False

    def __hash__(self):
        return hash(f"{self.position},{self.recipes_seen},{self.morids_seen},{self.rocky_remain},{self.cost}")

    def __str__(self) -> str:
        return f"\n cost:{self.cost}, \
                 \n position{self.position + 1} \
                 \n morids:{self.morids_seen} \
                 \n recipes:{self.recipes_seen} \
                  \n\n"  

## Class graph:

In [4]:
class Graph:
    def __init__(self, v :int) -> None:
        self.v = v
        self.edges = [[] for _ in range(v)]
        self.morids = None #make it dictionary
        self.recipes = []
        self.start_position = 0
        self.rocky_nodes = []

    def add_edge(self, u :int, v :int) -> None:
        self.edges[u].append(v)
        self.edges[v].append(u)

    def is_rocky(self, node :int) -> bool:
        return (node in self.rocky_nodes)

    def is_recipes(self, node :int) -> bool:
        return (node in self.recipes)

    def is_morid(self, node :int) -> bool:
        return (node in list(self.morids.keys()))

    def get_neighbors(self, s :State):
        return self.edges[s.position]


## Read input and create graph class

In [5]:
def create_graph(path :str) -> Graph:
    file = open(path, "r")
    n, m = map(int, file.readline().split(" "))
    g = Graph(n)

    for _ in range(m):
        u, v = map(int, file.readline().split(" "))
        g.add_edge(u - 1, v - 1)

    h = int(file.readline())
    rocky_nodes = list(map(int, file.readline().split(" ")))
    rocky_nodes = [x - 1 for x in rocky_nodes]
    g.rocky_nodes = rocky_nodes
    s = int(file.readline())

    morids = {}
    for _ in range(s):
        line = list(map(int, file.readline().split(" ")))
        line = [x - 1 for x in line]
        morids.update({line[0] : line[2:]})
    g.morids = morids
    
    g.start_position = (int(file.readline()) - 1)

    recipes = []
    for recipe in morids.values():
        recipes.extend(recipe)

    recipes = list(dict.fromkeys(recipes)) # Remove Duplicates From a Python List 
    g.recipes = recipes
    return g

g1 = create_graph("input.txt")
g2 = create_graph("input2.txt")
g3 = create_graph("input3.txt")
g4 = create_graph("input4.txt")


## Create initial state

In [6]:
def create_initial_state(g :Graph) -> State:
    s = State(g.rocky_nodes)
    s.position = g.start_position
    return s

initial_state1 = create_initial_state(g1)
initial_state2 = create_initial_state(g2)
initial_state3 = create_initial_state(g3)
initial_state4 = create_initial_state(g4)
 


 cost:0,                  
 position40                  
 morids:[]                  
 recipes:[]                   




## Transition module and goal state implementation:

In [7]:
counter = 0 # remember to initilize it to 0 in begenning

def reached_goal(s :State, g :Graph) -> bool:
    return len(g.morids) == len(s.morids_seen)

def is_in(p , q) -> bool:
    for i in q:
        if i not in p:
            return False
    return True
    
def transition(s :State, g :Graph, p :int) -> State:
    global counter
    counter += 1

    # if s.prev_state == None: #initial_state
    #     print(f"start : {s.position}\n")
    # else:
    #     print(f"{s.prev_state.position} => {s.position} \n")

    next_state = copy.deepcopy(s)
    next_state.prev_state = s
    next_state.cost += 1
    if s.rocky_remain != 0:
        next_state.rocky_remain -= 1
    else:
        next_state.position = p
        if g.is_rocky(p):
            next_state.rocky_remain = next_state.rocky_nodes[p]
            next_state.rocky_nodes[p] += 1

        elif(g.is_recipes(p) and p not in s.recipes_seen):
            next_state.recipes_seen.append(p)

        elif(g.is_morid(p) and p not in s.morids_seen and is_in(s.recipes_seen, g.morids[p])):
            next_state.morids_seen.append(p)
            
    # print(f"going to node {next_state.position + 1}, cost:{next_state.cost}")
    return next_state
    

## Print_path function:
to print path after finding it!

In [8]:
def print_path(final_state : State) -> None:
    print("PATH::")
    s = final_state
    while True:
        print(f"{s.position + 1} <== ", end = " ")
        s = s.prev_state
        if s.prev_state == None: #s is first node
            print(f"{s.position + 1} ")
            break
         

## Test function:

In [9]:
def test(g :Graph, method) -> None:
    global counter
    counter = 0
    start = time.time()
    path = method(g, create_initial_state(g))
    end = time.time()
    print(f"time:{end - start}")
    print_path(path)
    print(f"cost:{path.cost},")
    print(f"states:{counter},")

# BFS:

## BFS implementation:

### With QUEUE:

In [10]:
def bfs(g :Graph, initial_state :State) -> State:
    q = Queue()
    q.put(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:
        state = q.get()
        
        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")

        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.put(t)
                explored.add(t)
                if BFS_PRINT_PATH:
                    print(f"{t.prev_state.position + 1} => {t.position + 1}")
        if BFS_PRINT_PATH:
            print("\n")

### With List:

In [11]:
def bfs2(g :Graph, initial_state :State) -> State:
    q = []
    q.append(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:
        state = q.pop(0)
        #for test
        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")

        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.append(t)
                explored.add(t)
                # print(f"{t.prev_state.position + 1} => {t.position + 1}")
        # print("\n")

### With List comprehension:

In [13]:
def h(s :State) -> int:
    return len(s.recipes_seen) + len(s.morids_seen)

def bfs4(g :Graph, initial_state :State) -> State:
    max_h = 0
    q = Queue()
    q.put(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:

        while 1:
            state = q.get()

            if h(state) > max_h:
                max_h = h(state)
                break
            elif h(state) >= (max_h - 1):
                break

        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")
            
        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.put(t)
                explored.add(t)

## BFS Test:

### Test1:

In [14]:
test(g1, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
time:0.02753734588623047
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:287,


PATH:

    8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 8 | 287 | 0.026 |

### Test2:

In [43]:
# %%script false --no-raise-error
test(g2, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
time:62.594446659088135
PATH::
29 <==  7 <==  5 <==  23 <==  28 <==  23 <==  9 <==  24 <==  11 <==  3 <==  13 <==  19 <==  28 
cost:12,
states:473822,


PATH:

    29 <==  7 <==  5 <==  23 <==  28 <==  23 <==  9 <==  24 <==  11 <==  3 <==  13 <==  19 <==  28

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 12 | 473822 | 62 |

### Test3:

In [42]:
# %%script false --no-raise-error
test(g3, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
cost:14
cost:15
cost:16
cost:17
cost:18
cost:19
cost:20
time:51.75353932380676
PATH::
16 <==  12 <==  50 <==  25 <==  34 <==  9 <==  49 <==  47 <==  49 <==  43 <==  19 <==  1 <==  18 <==  41 <==  48 <==  30 <==  45 <==  31 <==  24 <==  38 <==  42 <==  40 
cost:21,
states:224527,


PATH:

    16 <==  12 <==  50 <==  25 <==  34 <==  9 <==  49 <==  47 <==  49 <==  43 <==  19 <==  1 <==  18 <==  41 <==  48 <==  30 <==  45 <==  31 <==  24 <==  38 <==  42 <==  40 

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 21 | 224527 | 52 |

### Test4:

In [17]:
test(g4, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
time:0.008675098419189453
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:92,


PATH:

    5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 8 | 92 | 0.03 |

# IDS:

## IDS implementation:

In [18]:
def dfs(state :State, step :int, g :Graph):
    if reached_goal(state, g):
        return state
    if step == 0:
        return None

    for neighbor in g.get_neighbors(state):
        next_state = transition(state, g, neighbor)
        result = dfs(next_state, step - 1, g)
        if result != None:
            return result
    return None

h = 0
def ids(g:Graph, initial_state :State) -> State:
    global h
    while True:
        h += 1
        print(f"h:{h}")

        result = dfs(initial_state, h, g)
        if result != None:
            return result
        print(f"cant find in h:{h}")

## IDS Tests:

### Test1:

In [19]:
test(g1, ids)

h:1
cant find in h:1
h:2
cant find in h:2
h:3
cant find in h:3
h:4
cant find in h:4
h:5
cant find in h:5
h:6
cant find in h:6
h:7
cant find in h:7
h:8
time:0.3609590530395508
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:4632,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| IDS | 8 | 4632 | 0.37 |

### Test2:

In [20]:
%%script false --no-raise-error
h = 11
test(g2, ids)

### Test3

In [21]:
%%script false --no-raise-error
h = 20
test(g3, ids)

### Test4:

In [22]:
h = 0
test(g4, ids)

h:1
cant find in h:1
h:2
cant find in h:2
h:3
cant find in h:3
h:4
cant find in h:4
h:5
cant find in h:5
h:6
cant find in h:6
h:7
cant find in h:7
h:8
time:0.038872480392456055
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:580,


# ASTAR:

## A-Star search: 

### Implementation:

In [31]:
alpha = 1

def huristic(s :State) -> int:
    global alpha
    return -len(s.morids_seen) * alpha
    # return -(len(s.morids_seen) + len(s.recipes_seen)) * alpha

def a_star(g:Graph, initial_state :State) -> State:
    explored = []
    frountier = [initial_state, ]
    cost = 0

    while True:
        next_state = min(frountier, key = lambda s : huristic(s) + s.cost)
        frountier.remove(next_state)
        explored.append(next_state)

        if next_state.cost > cost:
            cost = next_state.cost
            print(f"cost:{cost}")

        if reached_goal(next_state, g):
            return next_state
        for neighbor in g.get_neighbors(next_state):
            t = transition(next_state, g, neighbor)
            if t not in explored and t not in frountier:
                frountier.append(t)

## Test A-Star

In [55]:
alpha = 1

### Test1

In [30]:
test(g1 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.010292768478393555
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:85,


### Test3:

In [ ]:
test(g3 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
cost:14
cost:15
cost:16
cost:17
cost:18
cost:19
cost:20
cost:21
time:123.51942801475525
PATH::
16 <==  12 <==  50 <==  25 <==  34 <==  9 <==  49 <==  47 <==  49 <==  43 <==  19 <==  1 <==  18 <==  41 <==  48 <==  30 <==  45 <==  31 <==  24 <==  38 <==  42 <==  40 
cost:21,
states:37900,


### Test4:

In [ ]:
test(g4 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.012925386428833008
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:55,


## Test Weighted A-Star1:
    let alpha = 5

In [32]:
alpha = 5

### Test1

In [33]:
test(g1 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.008862733840942383
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:85,


### Test2

In [34]:
test(g2 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
cost:14
time:1.0872013568878174
PATH::
28 <==  19 <==  13 <==  3 <==  11 <==  7 <==  18 <==  24 <==  9 <==  22 <==  29 <==  7 <==  5 <==  23 <==  28 
cost:14,
states:3296,


### Test3

In [35]:
test(g3 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
cost:14
cost:15
cost:16
cost:17
cost:18
cost:19
cost:20
cost:21
cost:22
cost:23
cost:24
time:4.888986587524414
PATH::
19 <==  1 <==  18 <==  41 <==  48 <==  30 <==  45 <==  12 <==  12 <==  16 <==  12 <==  50 <==  25 <==  34 <==  9 <==  49 <==  47 <==  27 <==  42 <==  38 <==  24 <==  31 <==  39 <==  18 <==  40 
cost:24,
states:6428,


### Test4

In [36]:
test(g4 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.004624128341674805
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:49,


## Test Weighted A-Star2:
    let alpha = 10

In [37]:
alpha = 10

### Test1

In [38]:
test(g1 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.00983428955078125
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:85,


### Test2

In [39]:
test(g2 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
cost:14
time:0.5959761142730713
PATH::
28 <==  19 <==  13 <==  3 <==  11 <==  7 <==  18 <==  24 <==  9 <==  22 <==  29 <==  7 <==  5 <==  23 <==  28 
cost:14,
states:1960,


### Test3

In [40]:
test(g3 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
cost:14
cost:15
cost:16
cost:17
cost:18
cost:19
cost:20
cost:21
cost:22
cost:23
cost:24
cost:25
cost:26
cost:27
cost:28
cost:29
cost:30
cost:31
cost:32
time:0.4231135845184326
PATH::
16 <==  12 <==  12 <==  15 <==  31 <==  24 <==  38 <==  36 <==  4 <==  2 <==  43 <==  19 <==  1 <==  18 <==  41 <==  48 <==  41 <==  10 <==  50 <==  25 <==  34 <==  9 <==  49 <==  47 <==  49 <==  9 <==  49 <==  12 <==  15 <==  31 <==  39 <==  18 <==  40 
cost:32,
states:1320,


### Test4

In [41]:
test(g4 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.0038061141967773438
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:49,


# Conclusion:

    Number OF States Watched:

|     | Test1 | Test2 | Test3 | Test4 |
| --- |  ---  |  ---  |  ---  |  ---  |
| BFS |   287   |   473822  |   224527  |  92 |
| IDS |    4632  |  --   |   --  | 580  |
| A-Star |   85   |  26764   |  37900   |  55 |
| alpha:5 |   85   |   3296  |   6428  |  49 |
| alpha:10 |   85   |   1960  |  1320   |  49 |

# EXTRA:

## PATH checker

In [ ]:
path_test2 = [19, 13, 3, 11, 24, 9, 23, 28, 23, 5, 7, 29]
path_test3 = [42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19, 43, 49, 47, 49, 9, 34, 25, 50, 12, 16]

def check_path(g :Graph, path) -> bool:
    print("all recipes:")
    print(g.recipes)
    print("all morids:")
    print(g.morids)
    state = create_initial_state(g)
    print(state)

    for i in path:
        state = transition(state, g, i-1)
        print(state)
    return reached_goal(state, g)

x = check_path(g2, path_test2)
print(x)

all recipes:
[2, 10, 12, 4, 23, 8]
all morids:
{27: [2, 10, 12], 28: [4], 6: [23, 8]}

 cost:0,                  
 position28                  
 morids:[]                  
 recipes:[]                   



 cost:1,                  
 position19                  
 morids:[]                  
 recipes:[]                   



 cost:2,                  
 position13                  
 morids:[]                  
 recipes:[12]                   



 cost:3,                  
 position3                  
 morids:[]                  
 recipes:[12, 2]                   



 cost:4,                  
 position11                  
 morids:[]                  
 recipes:[12, 2, 10]                   



 cost:5,                  
 position24                  
 morids:[]                  
 recipes:[12, 2, 10, 23]                   



 cost:6,                  
 position9                  
 morids:[]                  
 recipes:[12, 2, 10, 23, 8]                   



 cost:7,                  
 po

## Examples:

In [ ]:
x = [ [] for _ in range(5)]
print(x)

[[], [], [], [], []]


In [ ]:
a = [1, 2, 3]
b = [4 ,5, 6]
a.extend(b)
print(a)

[1, 2, 3, 4, 5, 6]


In [ ]:
# #how map works in python

# x = [1,2,3,4,5]

# def funcc(y):
#     return y - 1

# i = list(map(funcc, x))
# print(i)
# print(x)

In [ ]:
# # using global var inside function

# x = 12
# def func():
#     global x
#     x +=3

# func()
# print(x)

In [ ]:
# print("salam", end = " ")
# print("salam")
# print("salam")

In [ ]:
# ll = [1,2,3,4,5]
# print([x - 1 for x in ll ])

In [ ]:
# #queue in python:

# q = Queue()
# q.put(1)
# q.put(2)
# q.put(3)
# # print(1 in q)
# print(q.get())
# print(q.get())
# print(q.get())

In [ ]:
#how to see if elem in list x:

# x = [1,2,3]
# print(12 in x)

In [ ]:
#so None works!
# aa = None

# if aa == 12:
#     print("ok")
# else:
#     print("ok2")

In [ ]:
aa = [1,2,3]
bb = [3,2,1]

Counter(aa) == Counter(bb)

print(aa)
print (bb)

# a = 67 

# def func():
#     print(a)

# # x = [12,13,True,"salam"]
# # if x == "salam":
# #     print("yes")
# # print("no")

# # print(range(10))
# func()

# d = {
#     1 : [1,2,3],
#     2 : [1,2,3],
#     9 : [1,2,3],
#     "salam" : 78,
#     4 : [1,2,3],
# }
# d["salam"] += 100
# print(d)

# s = "salam balajan"
# print(s[2:])
# sss = "33"
# b = int(sss.split(" "))
# # print(a)
# print(b)
# type(b)

[1, 2, 3]
[3, 2, 1]


In [ ]:
# d = {
#     1 : [1,2,3],
#     2 : [1,2,3],
#     9 : [1,2,3],
#     4 : [1,2,3],
# }

# l = list(d.keys())
# print(l)
# print(type(l))
# x = [1,4,"salam", 7]
# print(dict.fromkeys(x, 0))
# len(d)

# for recipe in d.values():
#     print(recipe)
#     print(type(recipe))



In [ ]:
# How deep copy works in python:
# g = Graph(12)
# h = copy.deepcopy(g)
# print(g.v)
# h.v = 33
# print(g.v)